- Build base model for this
- Try Streamlit
- Look at/for other datasets
- Email Reda

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [ ]:
mean_mse_te = 0
for ttime, tsnapshot in enumerate(test_iter):
    pred,H_t,C_t = model(tsnapshot.x, tsnapshot.edge_index, H_t,C_t)
    mean_mse_te = mean_mse_te + torch.mean(vmse(pred,tsnapshot.y))
    preds.append(pred.detach().numpy())
    true.append(tsnapshot.y.detach().numpy())
    if ttime%batch_size == 0:
        H_t = None

In [2]:
G = pickle.load(open('graphs/graph.pkl','rb'))

In [3]:
for node in G.nodes:
    G.nodes[node]['delivered'] = np.sum(G.nodes[node]['delivered'])
    G.nodes[node]['factory issue'] = np.sum(G.nodes[node]['delivered'])
    G.nodes[node]['sales_order'] = np.sum(G.nodes[node]['delivered'])
    G.nodes[node]['production'] = np.sum(G.nodes[node]['production'])

G_processed = nx.convert_node_labels_to_integers(G)

In [4]:
edge_attr = np.zeros((len(G_processed.edges()),4))

for i,e in enumerate(G_processed.edges(data=True)):
    edge_attr[i] = np.array(list(e[2].values()),dtype=float)

In [5]:
node_features = np.zeros(len(G_processed.nodes))
node_targets = np.zeros(len(G_processed.nodes))
for i,n in enumerate(G_processed.nodes(data=True)):
    node_features[i] = n[1]['sales_order']
    node_targets[i] = n[1]['production']

In [6]:
from torch_geometric.data import Data
from torch_geometric.transforms import RandomNodeSplit
Graph = Data(
    x=node_features,
    edge_index= np.array(list(G_processed.edges())).transpose(),
    edge_attr=edge_attr,
    y=node_targets
)

split = RandomNodeSplit('train_rest',num_splits=4,num_val=0,num_test=10)

a = split(Graph)

In [7]:
def nx_to_torch(g):
    g = nx.convert_node_labels_to_integers(g)
    node_features = np.zeros(len(g.nodes))
    node_targets = np.zeros(len(g.nodes))
    for i,n in enumerate(g.nodes(data=True)):
        node_features[i] = n[1]['sales_order']
        node_targets[i] = n[1]['production']

    edge_weights = np.zeros((len(g.edges()),4))

    for i,e in enumerate(g.edges(data=True)):
        edge_weights[i] = np.array(list(e[2].values()),dtype=float)
    return Data(
    x=np.reshape(node_features,(len(node_features),1)),
    edge_index= np.array(list(g.edges())).transpose(),
    edge_attr=edge_weights,
    y=node_targets
)

In [8]:
import random
node_set = set(G_processed.nodes)
other_node_set = set()
cv = []
while len(node_set) > 0:
    fold = random.sample(list(node_set),5)
    node_set = node_set.difference(fold)
    g_test = nx_to_torch(nx.subgraph(G_processed,fold))
    g_train = nx_to_torch(nx.subgraph(G_processed,list(node_set.union(other_node_set))))

    cv.append((g_train,g_test))
    other_node_set = other_node_set.union(set(fold))
    

In [9]:
cv

[(Data(x=[35, 1], edge_index=[2, 518], edge_attr=[518, 4], y=[35]),
  Data(x=[5, 1], edge_index=[2, 10], edge_attr=[10, 4], y=[5])),
 (Data(x=[35, 1], edge_index=[2, 504], edge_attr=[504, 4], y=[35]),
  Data(x=[5, 1], edge_index=[2, 10], edge_attr=[10, 4], y=[5])),
 (Data(x=[35, 1], edge_index=[2, 526], edge_attr=[526, 4], y=[35]),
  Data(x=[5, 1], edge_index=[2, 9], edge_attr=[9, 4], y=[5])),
 (Data(x=[35, 1], edge_index=[2, 508], edge_attr=[508, 4], y=[35]),
  Data(x=[5, 1], edge_index=[2, 10], edge_attr=[10, 4], y=[5])),
 (Data(x=[35, 1], edge_index=[2, 538], edge_attr=[538, 4], y=[35]),
  Data(x=[5, 1], edge_index=[2, 6], edge_attr=[6, 4], y=[5])),
 (Data(x=[35, 1], edge_index=[2, 529], edge_attr=[529, 4], y=[35]),
  Data(x=[5, 1], edge_index=[2, 9], edge_attr=[9, 4], y=[5])),
 (Data(x=[35, 1], edge_index=[2, 533], edge_attr=[533, 4], y=[35]),
  Data(x=[5, 1], edge_index=[2, 8], edge_attr=[8, 4], y=[5])),
 (Data(x=[35, 1], edge_index=[2, 519], edge_attr=[519, 4], y=[35]),
  Data(x=

In [10]:
from torch_geometric.nn.models import GraphSAGE
import torch
import torch.nn.functional as F


class GSAGE(torch.nn.Module):
    def __init__(self, hidden):
        super(GSAGE, self).__init__()
        self.recurrent = GraphSAGE(-1,hidden,2)
        self.linear = torch.nn.Linear(hidden, 1)

    def forward(self, x, edge_index, edge_attr):
        h = self.recurrent(x, edge_index, edge_attr)
        h = F.relu(h)
        h = self.linear(h)
        return h




In [11]:
cv[0][0].x.dtype

dtype('float64')

In [22]:
from tqdm import tqdm

model = GSAGE(hidden=10)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

train = cv[0][0]
for epoch in tqdm(range(400)):

    y_hat = model(torch.tensor(train.x,dtype=torch.float32), torch.tensor(train.edge_index,dtype=torch.int64), train.edge_attr)
    cost = torch.mean((y_hat-torch.tensor(train.y))**2)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()

100%|██████████| 400/400 [00:00<00:00, 1901.02it/s]


In [23]:
model.eval()
test = cv[0][1]
cost = 0

y_hat = model(torch.tensor(test.x,dtype=torch.float32), torch.tensor(test.edge_index,dtype=torch.int64), test.edge_attr)

cost = cost + torch.mean((y_hat-torch.tensor(test.y))**2)
cost = cost / 5
cost = cost.item()
print("RMSE: {:.4f}".format(cost**0.5))

RMSE: 1220.9285


In [24]:
test.y

array([6853.9632 ,  211.72512, 2147.45076,    0.     ,  222.65   ])

In [25]:
y_hat

tensor([[ 866.0465],
        [1477.3575],
        [1204.0852],
        [1045.1263],
        [ 884.9025]], grad_fn=<AddmmBackward0>)